# Fine-tuning Neural Network Hyperparameters

 the flexilibility of neural networks is also one of it's main draw backs i.e there are so many hyperparameters to tweak i.e the number of layers, the number of neurons, the type of activation function to us in each layer, the weight initialization logic, the type of optimizers to use, its learning rate, the batch size and more

 **how do you you know what combination to hyperparameters is the best for your task?**

* one option is to convert the keras model to a scikit-learn estimator and then use `GridSearchCV` and `RandomizedSearchCV` to finetune the hyperparameters.
* alternatively is using the `Keras Tuner ` library whixh is a hyperparameter tuning library for Keras models. It offers several tuning strategies and it's highly customizable with an excellent integration with Tensor-board.

 

In [10]:
from sklearn.datasets import fetch_california_housing
import tensorflow as tf
# fetch the data
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
import tensorflow as tf

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

In [11]:
import keras_tuner as kt

def build_model(hp):
    # define the hyperparamters
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons = hp.Int("n_neurons", min_value=16, max_value=256)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2,
                             sampling="log")
    optimizer = hp.Choice("optimizer", values=["sgd", "adam"])

    if optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        
    # build the model
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())
    for _ in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))
    model.add(tf.keras.layers.Dense(10, activation="softmax"))
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
                  metrics=["accuracy"])
    return model
    


To do a basic random search, one can create `kt.RandomSearch` tuner passing the `build_model` function th the constructor, and call the the tuner's `search()` method:

In [12]:
random_search_tuner = kt.RandomSearch(
    build_model, objective="val_accuracy", max_trials=5, overwrite=True,
    directory="my_fashio_mnist", project_name="my_rnd_search", seed=42)

random_search_tuner.search(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))


Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.004392764996737242

Best val_accuracy So Far: 0.004392764996737242
Total elapsed time: 00h 00m 10s


Given the `objective` is set to `val_accurancy`, the tumer prefers models with a higher validation accuracy, so once the tuner has finished searching, one can get the best model as follows:

In [15]:
top3_models = random_search_tuner.get_best_models(num_models=3)
best_model = top3_models[0]
print(top3_models)

[<Sequential name=sequential, built=True>, <Sequential name=sequential, built=True>, <Sequential name=sequential, built=True>]


one can also call `get_best_hyperparameters()` to get the kt.HyperParameters of the best models:

In [20]:
top3_params = random_search_tuner.get_best_hyperparameters(num_trials=3)

top3_params[0].values


{'n_hidden': 5,
 'n_neurons': 25,
 'learning_rate': 0.0006562536901904111,
 'optimizer': 'sgd'}

Each tuner is guided by a so-called *oracle*: before each trial, the tuner asks the oracle to tell it what the next trial should be. The `RandomSearch` tuner uses a `RandomSearchOracle`, which is pretty basic: it just picks the next trial randomly, as we saw earlier. Since the oracle keeps track of all the trials, you can ask it to give you the best one, and you can display a summary of that trial:

In [21]:
best_trial = random_search_tuner.oracle.get_best_trials(num_trials=1)[0]

best_trial.summary()

Trial 0 summary
Hyperparameters:
n_hidden: 5
n_neurons: 25
learning_rate: 0.0006562536901904111
optimizer: sgd
Score: 0.004392764996737242


to show the best hyperparameters as well as the validation accuracy, you can also access all the metrics directly:

In [23]:
best_trial.metrics.get_last_value("val_accuracy")

0.004392764996737242